<h1>IAU projekt 1. fáza</h1>
<h3>Autori: Jakub Grúber(50%), Maria Fedosenya(50%)</h3>

In [110]:
import pandas as pd
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
from sklearn import preprocessing

pd.set_option('display.max_columns', 25) #nastavené kvôli sessions.csv, ktoré má 25 stĺpcov, aby sa dali všetky prezrieť

<h1>Načítanie dát zo súborov</h1>

In [111]:
prod_filepath = "zdrojove_data/product.csv"
products = pd.read_csv(prod_filepath, sep='\t')
products.head()

,code,location,store_name,product_ean
0,RU,Europe/Moscow,Kashira,8840337777778
1,SK,Europe/Bratislava,Skalica,9613080891331
2,PH,Asia/Manila,Antipolo,8545276183603
3,PK,Asia/Karachi,Larkana,9999281937499
4,CN,Asia/Shanghai,Hushitai,1423598805373


In [112]:
sess_filepath = "zdrojove_data/session.csv"
sessions = pd.read_csv(sess_filepath, sep='\t')
sessions.head()

,pct_scroll_move,screen_width,ack,pct_mouse_move,scroll_move_total_rel_distance,page_activity_duration,pct_doubleclick,pct_click,total_load_time,mouse_move_total_rel_distance,product_ean,pct_click_product_info,browser_name,screen_height,pct_wild_mouse,pct_mouse_click,pct_input,session_duration,wild_mouse_duration,session_start,pct_scroll_move_duration,pct_scrandom,pct_rage_click,session_id,user_id
0,44.92257,1366,1.0,8.79303,11.16238,5.45167,9.31170,10.35168,2.81337,7.79215,4142933212429,41.07557,chrome,1080,53.33923,10.65208,0.00646,116.78143,5.44105,2021-05-11 10:16:37,11.39851,60.96280,42.77314,1126926,2369
1,53.93169,1280,0.0,10.69797,13.20789,8.03161,11.33962,7.58887,0.11913,13.78957,6554259262213,52.52006,chrome,1080,69.70571,8.32826,0.07267,134.68835,9.51278,2023-07-07 17:38:03,9.48454,56.71155,30.83474,103955,1186
2,41.66225,lower,1.0,11.45911,9.16087,7.73080,9.19028,8.11378,2.60292,5.95620,3525947927861,38.74596,edge,768,72.87642,10.18567,0.02227,126.47138,8.10733,2019-07-24 12:15:27,9.51528,58.17433,39.30398,731099,1265
3,37.59937,lower,1.0,9.78386,6.38416,9.70961,7.42759,6.54811,6.55699,6.36864,9631384503705,62.74747,safari,600,54.03365,7.75790,0.05988,263.06076,9.31265,2022-06-10 17:52:37,4.91020,68.47422,46.14026,1379903,467
4,62.70254,higher,0.0,7.39008,6.09215,11.10614,10.94899,8.45322,0.85335,7.94163,2238487280389,41.10241,edge,600,60.07533,7.59956,1.45341,146.88323,12.60862,2020-09-07 15:03:13,10.63283,28.45915,42.41093,691237,594


In [113]:
user_filepath = "zdrojove_data/user.csv"
users = pd.read_csv(user_filepath, sep='\t')
users.head()

,sex,user_id,current_location,address,job,name,race,mail,username,registration,birthdate
0,M,1270,"(Decimal('43.6199585'), Decimal('-39.027502'))",Sámova 235\n750 67 Police nad Metují,Lawyer,MUDr. Přemysl Říha,NaN,stastnyrostislav@seznam.cz,ales00,"08/23/2023, 00:00:00",1981-08-08
1,F,146,"(Decimal('67.5016935'), Decimal('169.135745'))",Döringstr. 1/8\n09217 Staßfurt,Augenoptiker,Verena Zirme,NaN,marinaschueler@aol.de,schweitzertoni,2022-07-02,2010-09-15
2,NaN,2178,"(Decimal('-83.0843785'), Decimal('35.417096'))","Stretto Surian, 1 Appartamento 5\n09071, Abbas...",NaN,Sabatino Salvemini,NaN,alberto76@virgilio.it,rviviani,26 Oct 2017,1957-07-10
3,NaN,1211,"(Decimal('85.896300'), Decimal('-141.411207'))",岡山県我孫子市上広谷4丁目16番1号 上野桜木パーク629,NaN,青木 稔,NaN,nishimuraasuka@yahoo.com,fmatsumoto,22 Feb 2020,1961-03-02
4,F,1678,"(Decimal('71.468916'), Decimal('-69.232181'))",USS Nelson\nFPO AE 34617,NaN,Vicki Lopez,NaN,ghowell@yahoo.com,nreed,2022/02/14,NaN


<h1>Analýza štruktúry dát</h1>

Na zistenie zakladných informácii o tabuľkách sme použili metódu `info()`, a keďže funkcia `describe()` počíta hodnoty na základe číselných údajov, využijeme ju len vtedy, ak to bude mať zmysel. <br>

Tabuľka **users** má 2588 záznamov, 11 stĺpcov s nasledujúcimi charakteristikami:
<ul>
<li>sex - pohlavie, F/M alebo NaN</li>
<li>user_id - identifikačné číslo používateľa, typu integer</li>
<li>current_location - je miesto, kde sa používateľ nachádza</li>
<li>address - adresa používateľa</li>
<li>job - zamestnanie používateľa<</li>
<li>name - meno používateľa</li>
<li>race - rasa používateľa</li>
<li>mail - email, ktorý  užívateľ používa</li>
<li>username - používateľské meno</li>
<li>registration - dátum registrície</li>
<li>birthdate - dátum narodenia</li>
</ul>

In [114]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2588 entries, 0 to 2587
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sex               1294 non-null   object
 1   user_id           2588 non-null   int64 
 2   current_location  2459 non-null   object
 3   address           2200 non-null   object
 4   job               1035 non-null   object
 5   name              2588 non-null   object
 6   race              776 non-null    object
 7   mail              2588 non-null   object
 8   username          2588 non-null   object
 9   registration      2588 non-null   object
 10  birthdate         1423 non-null   object
dtypes: int64(1), object(10)
memory usage: 222.5+ KB


Tabuľka **products** má 14912 záznamov a 4 atribúty:
<ul>
<li>code - kód štátu</li>
<li>location - miesto</li>
<li>store_name - názov obchodu</li>
<li>product_ean - EAN kód produktu, typ integer</li>
</ul>

In [115]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14912 entries, 0 to 14911
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         14900 non-null  object
 1   location     14912 non-null  object
 2   store_name   14912 non-null  object
 3   product_ean  14912 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 466.1+ KB


Tabuľka **sessions** má 11043 záznamov, 25 stĺpcov. Pri tejto tabuľke uvádzame aj `describe()` pretože má veľa číselných údajov, ktorým má zmysel uvádzať štatistické údaje.

In [116]:
sessions.info()
sessions.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11043 entries, 0 to 11042
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   pct_scroll_move                 11032 non-null  float64
 1   screen_width                    11043 non-null  object 
 2   ack                             11043 non-null  float64
 3   pct_mouse_move                  11032 non-null  float64
 4   scroll_move_total_rel_distance  11032 non-null  float64
 5   page_activity_duration          11032 non-null  float64
 6   pct_doubleclick                 11032 non-null  float64
 7   pct_click                       11032 non-null  float64
 8   total_load_time                 11043 non-null  float64
 9   mouse_move_total_rel_distance   11031 non-null  float64
 10  product_ean                     11043 non-null  int64  
 11  pct_click_product_info          11032 non-null  float64
 12  browser_name                    

,pct_scroll_move,ack,pct_mouse_move,scroll_move_total_rel_distance,page_activity_duration,pct_doubleclick,pct_click,total_load_time,mouse_move_total_rel_distance,product_ean,pct_click_product_info,pct_wild_mouse,pct_mouse_click,pct_input,session_duration,wild_mouse_duration,pct_scroll_move_duration,pct_scrandom,pct_rage_click,session_id,user_id
count,11032.000000,11043.000000,11032.000000,11032.000000,11032.000000,11032.000000,11032.000000,11043.000000,11031.000000,1.104300e+04,11032.000000,11032.000000,11032.000000,11032.000000,11043.000000,11032.000000,11032.000000,11032.000000,11032.000000,1.104300e+04,11043.000000
mean,51.598736,0.549217,9.956045,9.433332,9.048019,9.791655,9.009297,3.983034,8.116635,4.976341e+12,51.367468,50.746666,9.037040,0.387147,150.120858,9.027668,9.801604,48.827050,51.134057,7.476908e+05,1253.587612
std,12.603880,0.497594,1.996056,3.370660,2.200975,2.080895,1.001416,2.820560,2.147599,2.898641e+12,12.732629,12.964515,1.001694,1.987929,60.395237,2.341253,1.961457,13.833425,13.030987,4.321667e+05,722.973175
min,0.000000,0.000000,2.165200,0.000000,2.812950,2.268070,5.214400,0.003570,1.228680,1.234528e+09,0.000000,0.000000,5.272990,0.000000,-47.247120,0.648860,2.587140,0.000000,0.000000,1.249000e+03,0.000000
25%,43.227652,0.000000,8.746925,6.725008,7.308185,8.397730,8.342118,1.908750,6.593985,2.482877e+12,42.752547,42.035492,8.358828,0.014138,108.955965,7.257557,8.613705,39.479665,42.549657,3.694335e+05,630.000000
50%,51.492390,1.000000,10.229970,9.014625,9.046810,10.061465,9.013655,3.334810,7.980590,4.969185e+12,51.327285,50.541550,9.042620,0.051710,150.993030,9.134175,10.177305,48.826865,51.024950,7.517470e+05,1255.000000
75%,60.105700,1.000000,11.381840,12.184798,10.750452,11.278697,9.679718,5.354640,9.649370,7.486987e+12,59.910635,59.323082,9.707045,0.249630,190.924920,10.824945,11.195578,57.866447,59.887293,1.121034e+06,1879.000000
max,100.000000,1.000000,15.912470,20.349730,16.572150,16.180380,12.943650,23.487790,15.791510,9.997623e+12,100.000000,100.000000,12.965960,100.000000,366.445590,16.845850,15.529330,100.000000,100.000000,1.499931e+06,2499.000000


<h1>Identifikácia problémov v dátach s prvotným riešením </h1>
Pred ďalšou prácou s dátami, je vhodné si ich upraviť do vhodného formátu a zbaviť sa redundantných a duplicitných informácii. <br>

Najprv skontrolujeme, či máme nadbytočné záznamy, ktorých sa vieme zbaviť. Zistíme, či sú niektoré riadky duplicitné. 

In [117]:
users_duplicates = users[users.duplicated()]
len(users_duplicates)

0